## Loading the Training Model

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load the historical dataset to train the model
historical_data = pd.read_csv('historical_farm_data_tamil_nadu.csv')

# Encode categorical features
label_encoders = {}
for column in ['crop_type', 'growth_stage', 'soil_type']:
    le = LabelEncoder()
    historical_data[column + '_encoded'] = le.fit_transform(historical_data[column])
    label_encoders[column] = le

# Select features and target variable
features = ['soil_moisture', 'temperature', 'humidity', 'rain_forecast', 
            'crop_type_encoded', 'growth_stage_encoded', 'soil_type_encoded']
X = historical_data[features]
y = historical_data['water_demand']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Load new farm data for prediction
new_farms_data = pd.read_csv('five_farms_data.csv')

# Encode categorical features for new data
for column in ['crop_type', 'growth_stage', 'soil_type']:
    new_farms_data[column + '_encoded'] = label_encoders[column].transform(new_farms_data[column])

# Prepare features for new data
X_new = new_farms_data[features]
X_new_scaled = scaler.transform(X_new)


## Predicting the Required water demand for the Five Farms 

In [20]:

new_farms_data['predicted_water_demand'] = model.predict(X_new_scaled)


## Defining the PRIOROTY for Each Farms

In [22]:
def calculate_priority_score(row):
    crop_sensitivity = {
        'rice': 0.9, 'millets': 0.8, 'maize': 0.7, 'sugarcane': 0.85, 'cotton': 0.6,
        'tobacco': 0.75, 'chickpeas': 0.65, 'groundnut': 0.7, 'banana': 0.8, 'pulses': 0.7,
        'turmeric': 0.75, 'onion': 0.7, 'tomato': 0.75, 'brinjal': 0.7, 'carrot': 0.65,
        'cabbage': 0.65, 'cauliflower': 0.65, 'beans': 0.7, 'peas': 0.7, 'potato': 0.7,
        'sweet potato': 0.7, 'yam': 0.7, 'barley': 0.65, 'wheat': 0.7, 'sesame': 0.65,
        'sunflower': 0.7, 'pearl millet': 0.65, 'mustard': 0.65, 'soybean': 0.65, 'rapeseed': 0.65,
        'fennel': 0.7, 'coriander': 0.7, 'cumin': 0.7, 'anise': 0.7, 'cardamom': 0.75
    }
    sensitivity_score = crop_sensitivity.get(row['crop_type'], 0.5)
    predicted_water_demand = row['predicted_water_demand']
    soil_moisture_score = row['soil_moisture']
    land_size_score = row['land_size']
    rain_forecast_score = row['rain_forecast']
    
    # Calculate priority score
    priority_score = (0.4 * predicted_water_demand +
                      0.2 * soil_moisture_score +
                      0.2 * sensitivity_score +
                      0.1 * land_size_score -
                      0.4 * rain_forecast_score)
    
    return priority_score


# Calculate priority scores
new_farms_data['priority_score'] = new_farms_data.apply(calculate_priority_score, axis=1)


## Saving the Results

In [24]:
new_farms_data.to_csv('five_farms_priority.csv', index=False)
print("Priority scores for the five farms have been calculated and saved to 'five_farms_priority.csv'.")

Priority scores for the five farms have been calculated and saved to 'five_farms_priority.csv'.
